In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim
from gensim.models.phrases import Phrases, Phraser
import numpy as np

In [2]:
!pip install pyLDAvis==3.4.1

  Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)
  Using cached funcy-2.0-py2.py3-none-any.whl (30 kB)


In [9]:
#!pip install pandas==1.5.1

In [4]:
df = pd.read_csv('clean_review_all.csv')
df

,at,content,score,userName,contentp,contentp_clean,text_length
0,2023-09-21 14:31:30,Sering muncul peringatan 'anda tidak terhubung...,5,Wilson Nts,sering muncul peringatan anda tidak terhubung ...,sering muncul ingat tidak hubung jaring orbit ...,238
1,2023-09-28 00:14:40,"Masih sering DC. Tiba2 g ada koneksi, lampu in...",1,Saifullah fil,masih sering dc tiba ada koneksi lampu indokat...,sering dc tiba koneksi lampu indokator merah e...,226
2,2023-09-24 13:44:07,Bapuk. Lemot. Padahal sudah di Taru dekat jend...,1,allan afriyadi,bapuk lemot padahal sudah di taru dekat jendel...,jelek lambat padahal taru dekat jendela lantai...,269
3,2023-09-18 21:58:35,"Sering muncul tulisan ""tidak terhubung ke .......",3,Suryokusumo Risdika Rizki,sering muncul tulisan tidak terhubung ke nama ...,sering muncul tulis hubung nama wifi padahal j...,240
4,2023-10-02 21:40:00,Sudah beli model yg harganya sampai 2 jutaan. ...,3,Anton Karnadi,sudah beli model yg harganya sampai jutaan awa...,beli model harga juta awal kencang tahun pakai...,147
...,...,...,...,...,...,...,...
3017,2023-02-06 16:52:51,Good,4,Prinstony Bella,good,good,4
3018,2023-05-01 18:30:41,Semngat,4,Agus Adekaputra,semngat,semngat,7
3019,2023-01-18 22:03:06,🙏🙏🙏🙏,5,Meki Mote,NaN,NaN,0
3020,2023-04-02 18:06:26,👍👍👍👍,5,Bertus Soen,NaN,NaN,0


In [5]:
dfa = df[['contentp_clean']]
dfa = dfa.dropna()
dfa

,contentp_clean
0,sering muncul ingat tidak hubung jaring orbit ...
1,sering dc tiba koneksi lampu indokator merah e...
2,jelek lambat padahal taru dekat jendela lantai...
3,sering muncul tulis hubung nama wifi padahal j...
4,beli model harga juta awal kencang tahun pakai...
...,...
3014,lelettt
3016,oke
3017,good
3018,semngat


In [14]:
from helper import *
import warnings
warnings.filterwarnings('ignore')
import pandas as pd #a library to make the data more structured

ModuleNotFoundError: No module named 'helper'

In [22]:
text = dfa['contentp_clean']
text_list =  [i.split() for i in text]
print(len(text_list))
print(text_list)

3000
[['sering', 'muncul', 'ingat', 'tidak', 'hubung', 'jaring', 'orbit', 'kalau', 'bisa', 'klik', 'perangkat', 'hubung', 'langsung', 'layar', 'blank', 'putih', 'mau', 'isi', 'ulang', 'pulsa', 'bingung', 'aplikasi', 'sering', 'blank', 'edit', 'kirim', 'keluh', 'via', 'email', 'direspon', 'tim', 'orbit', 'sangat', 'baik', 'diupdate', 'aplikasi', 'normal'], ['sering', 'dc', 'tiba', 'koneksi', 'lampu', 'indokator', 'merah', 'enak', 'youtube', 'pakai', 'orbit', 'tiba', 'pulsa', 'hp', 'habis', 'gara', 'orbit', 'dc', 'niat', 'hemat', 'jadi', 'mahal', 'warbiasa', 'baik', 'nih', 'ada', 'masalah', 'kuota', 'bayar', 'posisi', 'bagus', 'indikator', 'jaring', 'full', 'bar', 'emang', 'masalah', 'layan'], ['jelek', 'lambat', 'padahal', 'taru', 'dekat', 'jendela', 'lantai', 'tapi', 'tetap', 'lambat', 'nonton', 'youtube', 'buffering', 'nonton', 'netflix', 'buffering', 'parah', 'gitu', 'sering', 'disconnect', 'gara', 'gara', 'pasword', 'salah', 'padahal', 'bener', 'masukin', 'password', 'nya', 'bahkan'

In [23]:
import gensim
#Create Bigram & Trigram Models
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(text_list, min_count=10)
trigram = Phrases(bigram[text_list])
for idx in range(len(text_list)):
    for token in bigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)
    for token in trigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)

In [24]:
from gensim import corpora, models
# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(text_list)
dictionary.filter_extremes(no_below=5, no_above=0.2)
#no_below (int, optional) – Keep tokens which are contained in at least no_below documents.
#no_above (float, optional) – Keep tokens which are contained in no more than no_above documents (fraction of total corpus size, not an absolute number).
print(dictionary)

Dictionary<751 unique tokens: ['aplikasi', 'baik', 'bingung', 'bisa', 'blank']...>


In [25]:
#https://radimrehurek.com/gensim/tut1.html
#build corpus
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
#The function doc2bow converts document (a list of words) into the bag-of-words format
'''The function doc2bow() simply counts the number of occurrences of each distinct word,
converts the word to its integer word id and returns the result as a sparse vector.
The sparse vector [(0, 1), (1, 1)] therefore reads: in the document “Human computer interaction”,
the words computer (id 0) and human (id 1) appear once;
the other ten dictionary words appear (implicitly) zero times.'''
print(len(doc_term_matrix))
print(doc_term_matrix[100])
tfidf = models.TfidfModel(doc_term_matrix) #build TF-IDF model
corpus_tfidf = tfidf[doc_term_matrix]

3000
[(0, 1), (1, 1), (6, 1), (7, 1), (10, 1), (17, 5), (27, 1), (44, 2), (64, 1), (66, 1), (95, 1), (102, 2), (110, 3), (118, 1), (155, 2), (157, 2), (173, 1), (187, 4), (221, 1), (295, 2), (344, 1), (426, 1), (475, 1), (551, 3), (552, 2), (553, 1)]


## Coherence Graph

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
#function to compute coherence values
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=100)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
start=1
limit=21
step=1
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf,
                                                        texts=text_list, start=start, limit=limit, step=step)
#show graphs
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
coherence_values

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 6))

In [ ]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=3) #num topic menyesuaikan hasil dari coherence value paling tinggi
for idx, topic in model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
import pandas as pd
top_words_per_topic = []
for t in range(model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in model.show_topic(t, topn = 10)])
#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")
df = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word','P']).to_csv("top_words2.csv")
print(df)

In [ ]:
dictionary

In [ ]:
corpus_tfidf

In [ ]:
pyLDAvis.__version__

In [ ]:
!pip install pyLDAvis==3.4.1

In [ ]:
import gensim
import pyLDAvis.gensim;pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary)
print(data)
pyLDAvis.save_html(data, 'lda-gensim.html')

In [ ]:
model.save('model')

## Mapping Topic

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
texts = []
for i in dfa['contentp_clean']:
  tokens = tokenizer.tokenize(i)
  texts.append(tokens)

In [ ]:
# load model
ldamodel = gensim.models.ldamodel.LdaModel.load('model')
# turn our tokenized documents into a id <-> term dictionary
dictionary = ldamodel.id2word

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

def format_topics_sentences(ldamodel=None, corpus=corpus, texts=texts):
  # Init output
  sent_topics_df = pd.DataFrame()

  # Get main topic in each document
  for i, row_list in enumerate(ldamodel[corpus]):
    row = row_list[0] if ldamodel.per_word_topics else row_list
    print(row)
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    # Get the Dominant topic, Perc Contribution and Keywords for each document
    for j, (topic_num, prop_topic) in enumerate(row):
      if j == 0:
        wp = ldamodel.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
      else:
        break
  sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

  # Add original text to the end of the output
  contents = pd.Series(texts)
  sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
  return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic

In [ ]:
df_dominant_topic['contentp_clean'] = df_dominant_topic['Text'].apply(' '.join)
df_dominant_topic

In [ ]:
topic_mapping = {
    0: "Harga Mahal",
    1: "Aplikasi Error",
    2: "Pelayanan",
    # Add mappings for all your topics
}

In [ ]:
df_dominant_topic['Topic'] = df_dominant_topic['Dominant_Topic'].map(topic_mapping)

In [ ]:
list(df_dominant_topic)

In [ ]:
df_dominant_topic = df_dominant_topic[[
 'Dominant_Topic',
 'Topic_Perc_Contrib',
 'Keywords',
 'contentp_clean',
 'Topic']]
df_dominant_topic

In [ ]:
topic_percentages = df_dominant_topic['Topic'].value_counts(normalize=True) * 100

In [ ]:
import matplotlib.pyplot as plt

topic_percentages.plot(kind='bar')
plt.xlabel('Dominant Topic')
plt.ylabel('Percentage')
plt.title('Percentage of Topics')
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

unique_topics = df_dominant_topic['Topic'].unique()

for topic in unique_topics:
    text = ' '.join(df_dominant_topic[df_dominant_topic['Topic'] == topic]['contentp_clean'])
    wordcloud = WordCloud(width=800, height=400).generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for Topic {topic}")
    plt.show()